In [ ]:
import requests
import json
API     = "https://leitlinien-api.awmf.org/v1/search"
API_KEY = "MkI5Y1VIOEJ0ZGpoelNBVXRNM1E6WVFld0pBUF9RLVdJa012UHVPTmRQUQ=="

s = requests.Session()
s.get("https://register.awmf.org/de/suche", headers={
    "User-Agent": "Mozilla/5.0"
})

s.headers.update({
    "Accept":           "application/json, text/plain, */*",
    "X-Requested-With": "XMLHttpRequest",
    "Api-Key":          API_KEY,
    "Origin":           "https://register.awmf.org",
    "Referer":          "https://register.awmf.org/de/suche",
    "User-Agent":       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"
})

def fetch_json(offset, limit=20):
    params = {"sorting":"relevance","limit":limit,"offset":offset,"lang":"de"}
    r = s.get(API, params=params)
    r.raise_for_status()
    return r.json()

try:
    with open("awmf.json", "r") as f:
        results = json.load(f)
except FileNotFoundError:
    results = fetch_json(0)
    with open("awmf.json", "w") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

In [69]:
import os
import time
from tenacity import retry, stop_after_attempt, wait_exponential, RetryError
import urllib.parse


@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1.5, min=3, max=20))
def fetch_pdf(url, base="https://register.awmf.org/assets/guidelines/"):
    r = s.get(urllib.parse.urljoin(base, url))
    r.raise_for_status()
    return r


for guideline in results["records"]:
    for link in guideline["links"]:
        if link.get("href"):
            url = link["href"]
        elif link.get("media"):
            url = link["media"]
        if not url or not url.endswith(".pdf") or os.path.exists(os.path.join("pdfs", url)):
            continue
        print(url)
        try:
            pdf = fetch_pdf(url)
        except RetryError as e:
            print(f"Error fetching {url}: {e}")
            continue
        target_path = os.path.join("pdfs", url)
        # Verzeichnis anlegen, falls noch nicht vorhanden:
        os.makedirs(os.path.dirname(target_path), exist_ok=True)
        with open(target_path, "wb") as f:
            f.write(pdf.content)
        time.sleep(5)

fileadmin/user_upload/Leitlinien/006_D_Ges_fuer_Kinderchirurgie/006-i_Lenkungsgruppe_2012.pdf
Error fetching fileadmin/user_upload/Leitlinien/006_D_Ges_fuer_Kinderchirurgie/006-i_Lenkungsgruppe_2012.pdf: RetryError[<Future at 0x134321ed0 state=finished raised HTTPError>]
fileadmin/user_upload/Leitlinien/030_D_Ges_fuer_Neurologie/030-036i_S1_Clusterkopfschmerz_trigeminoautonome_Kopfschmerzen_2015.pdf
Error fetching fileadmin/user_upload/Leitlinien/030_D_Ges_fuer_Neurologie/030-036i_S1_Clusterkopfschmerz_trigeminoautonome_Kopfschmerzen_2015.pdf: RetryError[<Future at 0x134543ad0 state=finished raised HTTPError>]
fileadmin/user_upload/Leitlinien/024_Ges_fuer_Neonatologie_und_Paediatrische_Intensivmedizin/024-008_S2_Bakterielle_Infektionen_des_Neugeborenen_02-2006_02-2011.pdf
Error fetching fileadmin/user_upload/Leitlinien/024_Ges_fuer_Neonatologie_und_Paediatrische_Intensivmedizin/024-008_S2_Bakterielle_Infektionen_des_Neugeborenen_02-2006_02-2011.pdf: RetryError[<Future at 0x1343201d0 st

In [3]:
len(results["records"])

1366

In [35]:
results["records"][0]

{'AWMFAssociationNumber': '050',
 'AWMFDetailPage': 'https://register.awmf.org/de/leitlinien/detail/050-001',
 'AWMFGuidelineClass': 'S3',
 'AWMFGuidelineID': '050-001',
 'AWMFGuidelineNumber': '001',
 'AWMFReleaseType': 'ohne',
 'description': '&nbsp;Die Leitlinie zeigt die Möglichkeiten, Effekte und Risiken der aktuellen evidenzbasierten Prävention und Therapie der Adipositas auf. Dabei werden die einzelnen Behandlungskomponenten separat dargestellt und bewertet.&nbsp;',
 'institutions': {'leading': [{'AWMFAssociationNumber': '050',
    'discipline': 'Adipositas; Deutsche Adipositas-Gesellschaft',
    'href': 'https://www.awmf.org/fachgesellschaften/deutsche-adipositas-gesellschaft-e-v-dag',
    'name': 'Deutsche Adipositas-Gesellschaft e.V. (DAG)'}],
  'participating': [{'AWMFAssociationNumber': '057',
    'discipline': 'Diabetes; Deutsche Diabetes-Gesellschaft',
    'href': 'https://www.awmf.org/fachgesellschaften/deutsche-diabetes-gesellschaft-e-v-ddg',
    'name': 'Deutsche Diabe

In [28]:

for record in results["records"][150:]:
    if int(record["validUntilDate"].split(".")[-1]) < 2025:
        if record["AWMFReleaseType"] not in ["Überarbeitung", "Klassen-Upgrade"]:
            display(record)
            break


{'AWMFAssociationNumber': '020',
 'AWMFDetailPage': 'https://register.awmf.org/de/leitlinien/detail/020-015',
 'AWMFGuidelineClass': 'S2k',
 'AWMFGuidelineID': '020-015',
 'AWMFGuidelineNumber': '015',
 'AWMFReleaseType': 'ohne',
 'description': '<p>Diese Leitlinie verfolgt das Ziel, konsentierte Aussagen zur Diagnostik und therapeutischen Strategien bei Patienten im prolongierten Weaning zu vermitteln.</p><p>In der Leitlinie werden Definitionen, Epidemiologie und Weaning-Kategorien, die zugrundeliegende Pathophysiologie, Strategien zur Prävention von prolongiertem Weaning, das gesamte Spektrum der verfügbaren Therapiestrategien, die Weaning-Einheit, die Überleitung in eine außerklinische Beatmung und schließlich Empfehlungen zu Therapieentscheidungen am Ende des Lebens bei prolongiertem bzw. erfolglosem Weaning abgehandelt.</p><p>Weitere Ziele der revidierten Leitlinie sind, den aktuellen Wissensstand zum Thema „Prolongiertes Weaning“ wissenschaftlich zu bewerten und auf Basis der Evi

In [11]:
record

{'AWMFAssociationNumber': '003',
 'AWMFDetailPage': 'https://register.awmf.org/de/leitlinien/detail/003-001',
 'AWMFGuidelineClass': 'S3',
 'AWMFGuidelineID': '003-001',
 'AWMFGuidelineNumber': '001',
 'AWMFReleaseType': 'Überarbeitung',
 'description': 'Das Ziel der Leitlinie ist, eine möglichst effektive Prophylaxe venöser Thromboembolien zu empfehlen.&nbsp;Die medizinische Effektivität eines Verfahrens wird bestimmt durch die Balance aus positiven Effekten (insbesondere Vermeidung thromboembolischer Komplikationen) und negativen Effekten (insbesondere Blutungskomplikationen und andere Nebenwirkungen).&nbsp;Aspekte der Effizienz und Ökonomie wurden mitberücksichtigt, waren aber nie primär entscheidungsbestimmend. Auch organisatorische und juristische Aspekte werden mitbetrachtet.',
 'institutions': {'leading': [{'AWMFAssociationNumber': '003',
    'discipline': '',
    'href': 'https://www.awmf.org/fachgesellschaften/deutsche-gesellschaft-fuer-chirurgie-e-v-dgch',
    'name': 'AWMF A

In [34]:
guidance_dict = {}
link_types = {}
for record in results["records"]:
    for link in record["links"]:
        if link["type"] not in link_types:
            link_types[link["type"]] = record["AWMFDetailPage"]
        if link["type"] == "practiceGuidance" and \
        link["title"] not in guidance_dict \
        and "patient" not in link["title"].lower() \
        and "evidenz" not in link["title"].lower():
            guidance_dict[link["title"]] = record["AWMFDetailPage"]
display(guidance_dict)

{'Appendix Methodenreport': 'https://register.awmf.org/de/leitlinien/detail/050-001',
 'Angaben zu Interessenkonflikten': 'https://register.awmf.org/de/leitlinien/detail/055-002',
 'Addendum 2023': 'https://register.awmf.org/de/leitlinien/detail/001-005',
 'Flowcharts': 'https://register.awmf.org/de/leitlinien/detail/001-012',
 'Addendum Dexmedetomidin': 'https://register.awmf.org/de/leitlinien/detail/001-012',
 'Appendix I: Recherche-Strategie': 'https://register.awmf.org/de/leitlinien/detail/001-016',
 'Dia-Version': 'https://register.awmf.org/de/leitlinien/detail/003-001',
 'Anmeldung Update': 'https://register.awmf.org/de/leitlinien/detail/003-001',
 'Implementierungshilfe': 'https://register.awmf.org/de/leitlinien/detail/013-018',
 'english version: allergen immunotherapy  in IgE-mediated allergic diseases': 'https://register.awmf.org/de/leitlinien/detail/061-004',
 '': 'https://register.awmf.org/de/leitlinien/detail/061-004',
 'English Version: diagnosis of drug hypersensitivity 

In [16]:
{record["AWMFReleaseType"] for record in results["records"]}

{'Klassen-Upgrade', 'Neuanmeldung', 'ohne', 'Überarbeitung'}

### Dataset info
* name: the official name of the guidline
* titleKeywords: a list of topics addressed in the guideline
* AWMFGuidelineClass: e.g. S3, S2k, S1
* AWMFDetailPage: link to guidline webpage on AWMF
* AWMFReleaseType is one of
  * Klassen-Upgrade: the guideline in question is being upgraded e.g. from S1 to S2k, this means that it undergoes reworking
  * Neuanmeldung: the guideline is currently being created, but is not yet available
  * ohne: standard
  * Überarbeitung: the guideline is currently being updated
* validUntilDate: may be in the past, whether it's being upadted is noted in AWMFReleaseType. We should include this in the metadata of the chunks, so that if there is conflicting information, the LLM should include the validUntilDate in its decision
* description: what is this guideline about?
* links
  * type
    * longVersion: the standard version of the guideline
    * practiceGuidance: may contain different kinds of documents, e.g. conflicts of interests or 'Addendum's, which can contain very useful information. There are a LOT of different types, so for now we might leave it out
    * relatedGuideline: references to other guidelines
    * patientGuideline: 
    * shortVersion
    * evidenceReport
    * relatedInformation


### Takeaways
* document selection
  * include links
    * with type 'longVersion'
    * with type 'practiceGuidance' if:
      * 'title' contains 'Addendum'
      * 'title' contains 'Schlüsselempfehlungen'
    addendums
  * leave out links with type 'guidelineReport', type 'shortVerison'
    * with type 'practiceGuidance' if:
      * 'title' contains 'Interessenkonflikt'
* include metadata such as 
  * titleKeywords
  * validUntilDate
  * lastEdit
  * description
  * AWMFGuidelineClass

In [45]:
document_list = []

for record in results["records"]:
    for link in record["links"]:
        if link["type"] == "longVersion":
            document_list.append(
                {
                    "title": link["title"],
                    "url": link["media"],
                    "type": record["AWMFReleaseType"],
                    "date": record["validUntilDate"],
                    "keywords": record["titleKeywords"],
                    "description": record["description"],
                    "guideline_title": record["name"],
                    "guideline_url": record["AWMFDetailPage"],
                    "guideline_id": record["AWMFGuidelineID"],
                    "validUntilDate": record["validUntilDate"],
                    "lastEdit": record["lastEdit"],
                    "AWMFGuidelineClass": record["AWMFGuidelineClass"],
                }
            )

len(document_list)

813

In [53]:
# for document in document_list, load the document from the pdfs directory and count the number of pages
import PyPDF2
import os
from tqdm import tqdm
from pathlib import Path
from PyPDF2 import PdfReader
from PyPDF2.errors import PdfReadError, DependencyError

def get_pdf_page_count(pdf_path):
    try:
        with open(pdf_path, "rb") as f:
            reader = PdfReader(f)
            return len(reader.pages)
    except (PdfReadError, FileNotFoundError, DependencyError):
        return 0


def get_pdf_page_counts(document_list):
    pdf_page_counts = {}
    for document in tqdm(document_list):
        pdf_path = Path("pdfs") / f"{document['url']}"
        if pdf_path.exists():
            page_count = get_pdf_page_count(pdf_path)
            pdf_page_counts[document["guideline_id"]] = page_count
        else:
            pdf_page_counts[document["guideline_id"]] = 0
    return pdf_page_counts

page_count_dict = get_pdf_page_counts(document_list)
total_page_count = 0
for page_count in page_count_dict.values():
    total_page_count += page_count
print(f"Total number of pages: {total_page_count}")


100%|██████████| 813/813 [00:12<00:00, 65.84it/s]

Total number of pages: 76190


In [52]:
page_count_dict = get_pdf_page_counts(document_list)
total_page_count = 0
for page_count in page_count_dict.values():
    total_page_count += page_count
print(f"Total number of pages: {total_page_count}")

100%|██████████| 813/813 [00:12<00:00, 66.42it/s] 

Total number of pages: 76190


In [56]:
url

'050_D_Adipositas_Ges/050-001a1_S3_Praevention-Therapie-Adipositas_2024-10.pdf'

In [40]:
# save the PDF
with open("050-001l_S3_Praevention-Therapie-Adipositas_2024-10.pdf", "wb") as f:
    f.write(guideline.content)